In [1]:
import pandas as pd
import spacy

In [7]:
%store -r w2v_model

In [2]:
arguments=pd.read_csv("arguments-validation.tsv", sep='\t')

In [4]:
labels=pd.read_csv("labels-validation.tsv", sep='\t')

In [5]:
# check for duplicate rows
duplicate_rows=arguments.duplicated().sum()
print(duplicate_rows)

0


In [6]:
# concatenate the two sets
validation_set = pd.concat([arguments, labels], axis=1)

In [9]:
# load the English language model
nlp = spacy.load("en_core_web_sm")

In [13]:
def preprocess_text(text):
    doc = nlp(text)
    # set lowercase
    processed_text = [token.text.lower()for token in doc]
    return processed_text

In [16]:
for id, row in validation_set.iterrows():
    print(row.Premise)

affirmative action helps with employment equity.
affirmative action can be considered discriminatory against poor whites
naturopathy is very dangerous for the most vulnerable people, like children and cancer patients. people use ineffective treatments and forgo proven cures, such as antibiotics or chemo, often resulting in death.
women shouldn't be in combat because they aren't as strong or fast as men and can be a weak link
once eradicated illnesses are returning due to people turning to naturopathy.
racial profiling is a preconceived idea of people that views an entire race as criminal
affirmative action is not fair to the rest of the people and knowing that should not be implemented.
flag burning is a freedom of speech and as such is protected under the constitution. it is an act of defiance that must continue to be allowed to protect freedoms for all.
we should subsidize stay-at-home dads because with them staying home, they more and likely would not have the same income level had 

Since from the 1576th row on we have all nans for the Premise we will drop these rows

In [20]:
validation_set = validation_set.iloc[:1576]

In [23]:
# we pre-process the arguments with spacy
validation_set["premise_tokenized"] = validation_set["Premise"].apply(preprocess_text)
validation_set.head()

,Argument ID,Conclusion,Stance,Premise,Argument ID,Self-direction: thought,Self-direction: action,Stimulation,Hedonism,Achievement,...,Conformity: rules,Conformity: interpersonal,Humility,Benevolence: caring,Benevolence: dependability,Universalism: concern,Universalism: nature,Universalism: tolerance,Universalism: objectivity,premise_tokenized
0,A26004,We should end affirmative action,against,affirmative action helps with employment equity.,A01001,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,"[affirmative, action, helps, with, employment,..."
1,A26010,We should end affirmative action,in favor of,affirmative action can be considered discrimin...,A01012,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,"[affirmative, action, can, be, considered, dis..."
2,A26016,We should ban naturopathy,in favor of,naturopathy is very dangerous for the most vul...,A02001,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,"[naturopathy, is, very, dangerous, for, the, m..."
3,A26024,We should prohibit women in combat,in favor of,women shouldn't be in combat because they aren...,A02002,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,"[women, should, n't, be, in, combat, because, ..."
4,A26026,We should ban naturopathy,in favor of,once eradicated illnesses are returning due to...,A02009,0,0,0,0,0,...,1,0,0,0,0,1,0,0,1,"[once, eradicated, illnesses, are, returning, ..."


In [24]:
# we generalize labels according to the following dictionary
general_dictionary={
    "Self-direction: thought":"Openness to change",
    "Self-direction: action":"Openness to change",
    "Stimulation":"Openness to change",
    "Hedonism":"Openness to change",
    "Achievement":"Self-Enhancement",
    "Power: dominance":"Self-Enhancement",
    "Power: resources":"Self-Enhancement",
    "Face":"Self-Enhancement",
    "Security: personal":"Conservation",
    "Security: societal":"Conservation",
    "Tradition":"Conservation",
    "Conformity: rules":"Conservation",
    "Conformity: interpersonal":"Conservation",
    "Humility":"Conservation",
    "Benevolence: caring":"Self-Transcendence",
    "Benevolence: dependability":"Self-Transcendence",
    "Universalism: concern":"Self-Transcendence",
    "Universalism: nature":"Self-Transcendence",
    "Universalism: tolerance":"Self-Transcendence",
    "Universalism: objectivity":"Self-Transcendence"
    
}

In [25]:
validation_set["Openness to change"]=validation_set["Self-direction: thought"]+validation_set["Self-direction: action"]+validation_set["Stimulation"]+validation_set["Hedonism"]

In [26]:
validation_set["Self-Enhancement"]=validation_set["Achievement"]+validation_set["Power: dominance"]+validation_set["Power: resources"]+validation_set["Face"]

In [27]:
validation_set["Conservation"]=validation_set["Security: personal"]+validation_set["Security: societal"]+validation_set["Tradition"]+validation_set["Conformity: rules"]+validation_set["Conformity: interpersonal"]+validation_set["Humility"]

In [28]:
validation_set["Self-Transcendence"]=validation_set["Benevolence: caring"]+validation_set["Benevolence: dependability"]+validation_set["Universalism: concern"]+validation_set["Universalism: nature"]+validation_set["Universalism: tolerance"]+validation_set["Universalism: objectivity"]

Now that the new features are created the original ones can be dropped

In [29]:
for key in general_dictionary:
    if key!=general_dictionary[key]:
        validation_set=validation_set.drop(key, axis=1)
    else:
        pass

In [30]:
validation_set

,Argument ID,Conclusion,Stance,Premise,Argument ID,premise_tokenized,Openness to change,Self-Enhancement,Conservation,Self-Transcendence
0,A26004,We should end affirmative action,against,affirmative action helps with employment equity.,A01001,"[affirmative, action, helps, with, employment,...",0,0,1,0
1,A26010,We should end affirmative action,in favor of,affirmative action can be considered discrimin...,A01012,"[affirmative, action, can, be, considered, dis...",0,0,0,1
2,A26016,We should ban naturopathy,in favor of,naturopathy is very dangerous for the most vul...,A02001,"[naturopathy, is, very, dangerous, for, the, m...",0,0,1,1
3,A26024,We should prohibit women in combat,in favor of,women shouldn't be in combat because they aren...,A02002,"[women, should, n't, be, in, combat, because, ...",1,0,0,0
4,A26026,We should ban naturopathy,in favor of,once eradicated illnesses are returning due to...,A02009,"[once, eradicated, illnesses, are, returning, ...",0,0,1,2
...,...,...,...,...,...,...,...,...,...,...
1571,E07272,We should end mass migration together.,in favor of,Mass migration is a phenomenon that causes dam...,D02068,"[mass, migration, is, a, phenomenon, that, cau...",1,1,0,0
1572,E07273,We should end mass migration together.,in favor of,Mass migration hurts the migrants themselves i...,D02069,"[mass, migration, hurts, the, migrants, themse...",0,1,1,2
1573,E07275,We should consider Russian interests in the EU...,in favor of,It is neither in the interests of the EU nor R...,D02070,"[it, is, neither, in, the, interests, of, the,...",1,0,2,1
1574,E07280,We should adopt an extension of the applicatio...,in favor of,Foreign Policy at the EU level should be based...,D03043,"[foreign, policy, at, the, eu, level, should, ...",0,0,2,1


In [31]:
# extract the generalized label columns and store them in a separate DataFrame
label_columns=validation_set[[
"Openness to change",
"Self-Enhancement",
"Conservation",
"Self-Transcendence"]]

# convert the label columns to a multi-class format (one-hot encoding)
validation_set['general_label'] = label_columns.apply(lambda row: row.to_list(), axis=1)
validation_set

,Argument ID,Conclusion,Stance,Premise,Argument ID,premise_tokenized,Openness to change,Self-Enhancement,Conservation,Self-Transcendence,general_label
0,A26004,We should end affirmative action,against,affirmative action helps with employment equity.,A01001,"[affirmative, action, helps, with, employment,...",0,0,1,0,"[0, 0, 1, 0]"
1,A26010,We should end affirmative action,in favor of,affirmative action can be considered discrimin...,A01012,"[affirmative, action, can, be, considered, dis...",0,0,0,1,"[0, 0, 0, 1]"
2,A26016,We should ban naturopathy,in favor of,naturopathy is very dangerous for the most vul...,A02001,"[naturopathy, is, very, dangerous, for, the, m...",0,0,1,1,"[0, 0, 1, 1]"
3,A26024,We should prohibit women in combat,in favor of,women shouldn't be in combat because they aren...,A02002,"[women, should, n't, be, in, combat, because, ...",1,0,0,0,"[1, 0, 0, 0]"
4,A26026,We should ban naturopathy,in favor of,once eradicated illnesses are returning due to...,A02009,"[once, eradicated, illnesses, are, returning, ...",0,0,1,2,"[0, 0, 1, 2]"
...,...,...,...,...,...,...,...,...,...,...,...
1571,E07272,We should end mass migration together.,in favor of,Mass migration is a phenomenon that causes dam...,D02068,"[mass, migration, is, a, phenomenon, that, cau...",1,1,0,0,"[1, 1, 0, 0]"
1572,E07273,We should end mass migration together.,in favor of,Mass migration hurts the migrants themselves i...,D02069,"[mass, migration, hurts, the, migrants, themse...",0,1,1,2,"[0, 1, 1, 2]"
1573,E07275,We should consider Russian interests in the EU...,in favor of,It is neither in the interests of the EU nor R...,D02070,"[it, is, neither, in, the, interests, of, the,...",1,0,2,1,"[1, 0, 2, 1]"
1574,E07280,We should adopt an extension of the applicatio...,in favor of,Foreign Policy at the EU level should be based...,D03043,"[foreign, policy, at, the, eu, level, should, ...",0,0,2,1,"[0, 0, 2, 1]"


In [33]:
# tranform the labels in binary encoding
binary_labels = []

for label in validation_set["general_label"]:
    new_label = []

    for value in label:
        if value > 0:
            new_label.append(1)
        else:
            new_label.append(0)

    binary_labels.append(new_label)


In [34]:
validation_set["general_label"] = binary_labels

In [35]:
validation_set.head()

,Argument ID,Conclusion,Stance,Premise,Argument ID,premise_tokenized,Openness to change,Self-Enhancement,Conservation,Self-Transcendence,general_label
0,A26004,We should end affirmative action,against,affirmative action helps with employment equity.,A01001,"[affirmative, action, helps, with, employment,...",0,0,1,0,"[0, 0, 1, 0]"
1,A26010,We should end affirmative action,in favor of,affirmative action can be considered discrimin...,A01012,"[affirmative, action, can, be, considered, dis...",0,0,0,1,"[0, 0, 0, 1]"
2,A26016,We should ban naturopathy,in favor of,naturopathy is very dangerous for the most vul...,A02001,"[naturopathy, is, very, dangerous, for, the, m...",0,0,1,1,"[0, 0, 1, 1]"
3,A26024,We should prohibit women in combat,in favor of,women shouldn't be in combat because they aren...,A02002,"[women, should, n't, be, in, combat, because, ...",1,0,0,0,"[1, 0, 0, 0]"
4,A26026,We should ban naturopathy,in favor of,once eradicated illnesses are returning due to...,A02009,"[once, eradicated, illnesses, are, returning, ...",0,0,1,2,"[0, 0, 1, 1]"


In [37]:
w2w_vectors = []
for arg in validation_set["premise_tokenized"]:
    seq = []

    for word in arg:
        if word in w2v_model.wv:
            seq.append(word)
        else:
            pass
        
    w2w_vectors.append(seq)
    

In [38]:
w2w_vectors

[['action', 'helps', 'with', 'employment', '.'],
 ['action', 'can', 'be', 'considered', 'against', 'poor'],
 ['is',
  'very',
  'dangerous',
  'for',
  'the',
  'most',
  'vulnerable',
  'people',
  ',',
  'like',
  'children',
  'and',
  'cancer',
  'patients',
  '.',
  'people',
  'use',
  'treatments',
  'and',
  'proven',
  'cures',
  ',',
  'such',
  'as',
  'or',
  ',',
  'often',
  'resulting',
  'in',
  'death',
  '.'],
 ['women',
  'should',
  "n't",
  'be',
  'in',
  'combat',
  'because',
  'they',
  'are',
  "n't",
  'as',
  'strong',
  'or',
  'fast',
  'as',
  'men',
  'and',
  'can',
  'be',
  'a',
  'weak'],
 ['once', 'illnesses', 'are', 'due', 'to', 'people', 'to', '.'],
 ['is',
  'a',
  'idea',
  'of',
  'people',
  'that',
  'views',
  'an',
  'entire',
  'race',
  'as',
  'criminal'],
 ['action',
  'is',
  'not',
  'fair',
  'to',
  'the',
  'rest',
  'of',
  'the',
  'people',
  'and',
  'that',
  'should',
  'not',
  'be',
  '.'],
 ['is',
  'a',
  'freedom',
  'of